In [8]:
pinecone_api_key = "pcsk_pPg7v_J3LPtdMeHLhHgpeLruH6dwN3dDPimrK7TVUkR8HFm8c6G6jAkFy2tmTNeb191F4"
pinecone_aws_arn = "arn:aws:secretsmanager:ap-southeast-2:116981794561:secret:Pinecone_API_key-FQbBoB"
pinecone_index_url = "https://movie-time-hk-un08756.svc.aped-4627-b74a.pinecone.io"

In [6]:
from common import *

In [9]:
pc = Pinecone(api_key=pinecone_api_key)
INDEX_NAME = "movie-time-hk"  # @param {type:"string"}
DISTANCE_METRIC = ("cosine")
EMBEDDING_DIMENSION_SIZE = 1024  # @param {"type":"number","placeholder":"768"}
CLOUD_PROVIDER = "aws"  # @param ["gcp", "aws", "azure"] {allow-input: true}
CLOUD_REGION = "us-east-1"  # @param {type:"string"}

NameError: name 'Pinecone' is not defined

In [ ]:
pc.create_index(
    name=INDEX_NAME,
    dimension=EMBEDDING_DIMENSION_SIZE,
    metric=DISTANCE_METRIC,
    spec=ServerlessSpec(cloud=CLOUD_PROVIDER, region=CLOUD_REGION),
    deletion_protection="disabled",
)

In [6]:
boto3.DEFAULT_SESSION = boto3.Session()

Running boto3 version: 1.36.26


In [ ]:
pinecone_secret_filename = "pinecone-secret.txt"
def get_or_create_secret(filename, secret_name, secret_value):
    if os.path.exists(filename):
        print(f'File {filename} already exists. Reading secret ARN from the file.')
        with open(filename, 'r') as file:
            secret_arn = file.read().strip()
            print(f'Secret ARN: {secret_arn}')
            return secret_arn  # Return the existing secret ARN

    # If the file does not exist, create the secret
    print(f'File {filename} does not exist. Creating secret.')
    secret_arn = create_secret(secret_name, secret_value)

    if secret_arn:
        # Write the secret ARN to the file
        with open(filename, 'w') as file:
            file.write(secret_arn)
            print(f'Secret ARN saved to {filename}.')

    return secret_arn

def create_secret(secret_name, secret_value):
    # Create a Secrets Manager client
    client = boto3.client('secretsmanager', region_name='ap-southeast-2')

    try:
        # Create the secret
        response = client.create_secret(
            Name=secret_name,
            SecretString=json.dumps(secret_value)
        )
        print(f'Secret created: {response["ARN"]}')
        return response["ARN"]
    except Exception as e:
        if e.response['Error']['Code'] == 'ResourceExistsException':
            print(f'Secret {secret_name} already exists.')
        else:
            print(f'Error creating secret: {e}')

# Define the secret name and value
secret_name = 'pineconeApiKey'
secret_value = {
    'apiKey': pinecone_api_key
}

# Create and save the secret
get_or_create_secret(pinecone_secret_filename, secret_name, secret_value)

In [5]:
def create_bucket(bucket_name):
    s3 = boto3.client('s3', region_name='ap-southeast-2')
    try:
        # Create the S3 bucket
        response = s3.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={
                'LocationConstraint': 'ap-southeast-2'
            }
        )
        print(f'Bucket created: {response["Location"]}')
        return response["Location"]
    except ClientError as e:
        if e.response['Error']['Code'] == 'BucketAlreadyOwnedByYou':
            print(f'Bucket {bucket_name} already exists and is owned by you.')
        else:
            print(f'Error creating bucket: {e}')

def upload_file_to_bucket(bucket_name, file_name):
    s3 = boto3.client('s3', region_name='ap-southeast-2')
    try:
        # Upload the file to the bucket
        s3.upload_file(file_name, bucket_name, file_name)
        print(f'File {file_name} uploaded to bucket {bucket_name}.')
    except ClientError as e:
        print(f'Error uploading file: {e}')

# Define the bucket name and the file name
now = datetime.now()
bucket_name = f'movie-time-data-{now.year}-{now.month}-{now.day}' 
file_name = f'movies_today_{now.year}_{now.month}_{now.day}.csv'

# Create the bucket
create_bucket(bucket_name)

# Upload the file to the bucket
upload_file_to_bucket(bucket_name, file_name)

NameError: name 'boto3' is not defined

In [ ]:
bedrock = boto3.client('bedrock-agent', region_name='ap-southeast-2')

knowledge_base_config = {
    "type": "VECTOR",
    "vectorKnowledgeBaseConfiguration": {
        "embeddingModelArn": "arn:aws:bedrock:ap-southeast-2::foundation-model/amazon.titan-embed-text-v2:0",
        'embeddingModelConfiguration': {
            'bedrockEmbeddingModelConfiguration': {
                'dimensions': 1024,
                # 'embeddingDataType': 'BINARY'
            }
        },

    },
}

storageConfiguration = {
    "type": "PINECONE",
    "pineconeConfiguration" : {
        "connectionString": pinecone_index_url,
        "credentialsSecretArn": pinecone_aws_arn,
        "fieldMapping": {
            "metadataField": "metadata",
            "textField": "text",
        }
    }
}
response = bedrock.create_knowledge_base(
    name="PineconKnowledgeBase2",
    description="Knowledge base using S3 and Pinecone",
    knowledgeBaseConfiguration=knowledge_base_config,
    roleArn="arn:aws:iam::116981794561:role/service-role/AmazonBedrockExecutionRoleForKnowledgeBase_xi4h1",
    storageConfiguration=storageConfiguration,
)
print(response)
knowledgeBase = response["knowledgeBase"]

In [32]:
create_ds_response = bedrock.create_data_source(
    name = "movie-time-data-2025-2-22",
    description = "data for movie time",
    knowledgeBaseId = knowledgeBase['knowledgeBaseId'],
    dataSourceConfiguration = {
        "type": "S3",
        "s3Configuration": {
            "bucketArn": "arn:aws:s3:::movie-time-data-2025-2-22"
        }
    },
)
ds = create_ds_response["dataSource"]

In [ ]:
bedrock.get_data_source(knowledgeBaseId = knowledgeBase['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

In [34]:
start_job_response = bedrock.start_ingestion_job(knowledgeBaseId = knowledgeBase['knowledgeBaseId'], dataSourceId = ds["dataSourceId"])

In [23]:
# Initialize the Bedrock Agent Runtime client
bedrock_agent_runtime = boto3.client('bedrock-agent-runtime', region_name='ap-southeast-2')
model = "Mistral Large (24.02)"
model_arn = f'arn:aws:bedrock:ap-southeast-2::foundation-model/mistral.mistral-large-2402-v1:0'

In [24]:
def ask_bedrock_llm_with_knowledge_base(query: str, model_arn: str, kb_id: str) -> str:
    response = bedrock_agent_runtime.retrieve_and_generate(
        input={
            'text': query
        },
        retrieveAndGenerateConfiguration={
            'type': 'KNOWLEDGE_BASE',
            'knowledgeBaseConfiguration': {
                'knowledgeBaseId': kb_id,
                'modelArn': model_arn
            }
        },
    )

    return response

In [42]:
query = "can you recommend some movie for today?"


response = ask_bedrock_llm_with_knowledge_base(query, model_arn, knowledgeBase['knowledgeBaseId'])
generated_text = response['output']['text']
citations = response["citations"]
contexts = []
for citation in citations:
    retrievedReferences = citation["retrievedReferences"]
    for reference in retrievedReferences:
        contexts.append(reference["content"]["text"])
print(f"---------- Generated using {model}:")
print(generated_text )
print(f'---------- The citations for the response generated by {model}:')
print(contexts)
print()

---------- Generated using Mistral Large (24.02):
Here are some movies screening today: "My Best Bet" at CANDY PARK BY CINEMA CITY (愉景新城) at 22:05, "Living in Two Worlds" at 百老匯電影中心 at 19:45, "The Sinking of the Lisbon Maru" at CGV Cinemas 澳門星皓廣場 at 18:15, "(G)I-DLE WORLD TOUR [iDOL] in CINEMAS" at K11 Art House at 17:15 and Festival Grand Cinema at 17:30, "LITTLE RED SWEET" at STAR Cinema at 17:00 and MCL 德福戲院 at 16:15, and "Ha Ha Ha Happy New Year!!" at 嘉禾黃埔 at 16:40.
---------- The citations for the response generated by Mistral Large (24.02):
[", threatre='百老匯電影中心')] Not today My Best Bet - [Screening(date=datetime.datetime(2025, 2, 22, 22, 5, 0, 83147), threatre='CANDY PARK BY CINEMA CITY (愉景新城)')] Not today Living in Two Worlds - [Screening(date=datetime.datetime(2025, 2, 22, 19, 45, 0, 83147), threatre='百老匯電影中心')] Not today Not today Not today Not today Not today Not today Not today Not today Not today Not today Not today Not today Not today Not today The Sinking of the Lisbon M

In [ ]:

def invoke_agent(agent_id, agent_alias_id, prompt, session_id):
    response = bedrock_agent_runtime.invoke_agent(
        agentId=agent_id,
        agentAliasId=agent_alias_id,
        sessionId=session_id,
        inputText=prompt
    )

    completion = ""
    for event in response.get('completion'):
        chunk = event['chunk']
        completion += chunk['bytes'].decode()

    return completion

# Example usage
agent_id = 'HH9XK3W3AY'
agent_alias_id = 'SMD0SLDZKR'
session_id = 'unique_session_id'
prompt = "What's the current time?"

response = invoke_agent(agent_id, agent_alias_id, prompt, session_id)
print(f"Agent response: {response}")
